In [1]:
import os
# Find the latest version of spark 3.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.2'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
#!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!wget -q https://downloads.apache.org/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz

!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:3 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://security.ubuntu.com/ubuntu bionic-securi

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-08-08 18:56:02--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  5.70MB/s    in 0.2s    

2021-08-08 18:56:02 (5.70 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [4]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Home_Improvement_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   48881148|R215C9BDXTDQOW|B00FR4YQYK|     381800308|SadoTech Model C ...|Home Improvement|          4|            0|          0|   N|                Y|          Four Stars|        good product| 2015-08-31|
|         US|   47882936|R1DTPUV1J57YHA|B00439MYYE|     921341748|iSpring T32M 3.2 ...|Home Improvement|          5|    

### Create DataFrames to match tables

In [5]:
from pyspark.sql.functions import to_date, count
# Read in the Review dataset as a DataFrame
df.groupBy("customer_id").count().collect()

[Row(customer_id=28377689, count=1),
 Row(customer_id=28258386, count=2),
 Row(customer_id=9967574, count=1),
 Row(customer_id=25153155, count=1),
 Row(customer_id=10088068, count=1),
 Row(customer_id=45657423, count=1),
 Row(customer_id=19021463, count=1),
 Row(customer_id=41413793, count=2),
 Row(customer_id=43789873, count=2),
 Row(customer_id=12406466, count=1),
 Row(customer_id=41045019, count=1),
 Row(customer_id=35535911, count=2),
 Row(customer_id=47108763, count=1),
 Row(customer_id=48113150, count=1),
 Row(customer_id=18201417, count=9),
 Row(customer_id=26079415, count=3),
 Row(customer_id=36114891, count=7),
 Row(customer_id=3712628, count=1),
 Row(customer_id=37499901, count=2),
 Row(customer_id=47321438, count=2),
 Row(customer_id=134748, count=1),
 Row(customer_id=35843463, count=1),
 Row(customer_id=52906355, count=1),
 Row(customer_id=50525389, count=1),
 Row(customer_id=31081382, count=1),
 Row(customer_id=49041516, count=2),
 Row(customer_id=20587872, count=2),
 Row(

In [7]:
# Create the customers_table DataFrame
#customers_df = df.groupby("customer_id").agg({""}).withColumnRenamed("", "customer_count")
customers_df = df.groupby("customer_id").count().withColumnRenamed("count", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   28377689|             1|
|   28258386|             2|
|    9967574|             1|
|   25153155|             1|
|   10088068|             1|
|   45657423|             1|
|   19021463|             1|
|   41413793|             2|
|   43789873|             2|
|   12406466|             1|
|   41045019|             1|
|   35535911|             2|
|   47108763|             1|
|   48113150|             1|
|   18201417|             9|
|   26079415|             3|
|   36114891|             7|
|    3712628|             1|
|   37499901|             2|
|   47321438|             2|
+-----------+--------------+
only showing top 20 rows



In [20]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id","product_title"]).dropDuplicates(["product_id"])
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|7800807606|CBconcept Brand H...|
|9178907519|IKEA - GAVIK Tabl...|
|B00002N70K|Kwikset 293 Small...|
|B00002N7BC|Westinghouse 6720...|
|B00002NAEP|Westinghouse One-...|
|B00002NAVH|Leviton L02-49686...|
|B00002NB2F|View-Pak UBIP2020...|
|B00004SQLR|Master Lock 1547D...|
|B00004WLKJ|Gardner Bender He...|
|B00004Y61P|Towing/Recovery S...|
|B00004YUN1|Leviton 6697-W An...|
|B00004Z4AV|3M 4.875 by 2.875...|
|B0000AHSA7|Pollenex DP1011 H...|
|B0000BYEF5|CONROS #PM00506 6...|
|B0000BYEF6|Lutron Electronic...|
|B0000CEZ7C|Northern Industri...|
|B0000DBIKH|Brita 42633 Fauce...|
|B0000DD0F2|Shop-Vac 905-03-0...|
|B0000DEZNE|Dale Adams Enterp...|
|B0000DI5II|GE 75 Watt 2 Pack...|
+----------+--------------------+
only showing top 20 rows



In [11]:
# Create the review_id_table DataFrame. 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id","customer_id","product_id","product_parent", to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R215C9BDXTDQOW|   48881148|B00FR4YQYK|     381800308| 2015-08-31|
|R1DTPUV1J57YHA|   47882936|B00439MYYE|     921341748| 2015-08-31|
| RFAZK5EWKJWOU|   44435471|B00002N762|      56053291| 2015-08-31|
|R2XT8X0O0WS1AL|   28377689|B000QFCP1G|     595928517| 2015-08-31|
|R14GRNANKO2Y2J|   50134766|B00WRCRKOI|     417053744| 2015-08-31|
|R2BLF9VYL24LCQ|   14066511|B00NIH88EW|     275395071| 2015-08-31|
|R1GI9UW5KJ671O|   15211046|B005B9CI96|     856617815| 2015-08-31|
|R2H5CEJN863M86|   14862498|B008L0OMWI|     125102494| 2015-08-31|
| R5PPDHFOZ3SMU|   23617292|B00P9FTC6O|     523110842| 2015-08-31|
| RE1L9IENKJJ7Y|   35820485|B00K6BQEHQ|     797306964| 2015-08-31|
|R3CZ0990QC2Z0H|   47162350|B008BYQCWM|     865874404| 2015-08-31|
|R3UMMD2IO29QSP|   31884789|B003BLHTOU|     183592595| 2015-08

In [12]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R215C9BDXTDQOW|          4|            0|          0|   N|                Y|
|R1DTPUV1J57YHA|          5|            0|          0|   N|                Y|
| RFAZK5EWKJWOU|          5|            0|          0|   N|                Y|
|R2XT8X0O0WS1AL|          5|            0|          0|   N|                Y|
|R14GRNANKO2Y2J|          5|            0|          0|   N|                Y|
|R2BLF9VYL24LCQ|          5|            1|          1|   N|                Y|
|R1GI9UW5KJ671O|          5|            0|          0|   N|                Y|
|R2H5CEJN863M86|          5|            0|          1|   N|                Y|
| R5PPDHFOZ3SMU|          5|            0|          0|   N|                Y|
| RE1L9IENKJJ7Y|          1|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [13]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.co74liebdjcj.us-east-1.rds.amazonaws.com:5432/postgres"
config = {"user":"postgres", 
          "password": "Shashank1!", 
          "driver":"org.postgresql.Driver"}

In [14]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [21]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [22]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [23]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)